# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#read csv
city_weather = pd.read_csv("city_weather.csv")
city_weather


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mana,5.6592,-53.7767,77.25,83,96,11.74,GF,1611025426
1,1,Barrow,71.2906,-156.7887,-2.20,78,90,20.71,US,1611025426
2,2,Cape Town,-33.9258,18.4232,68.00,88,0,10.36,ZA,1611025426
3,3,Atasū,48.6814,71.6433,14.02,87,93,5.12,KZ,1611025426
4,4,Vaini,-21.2000,-175.2000,86.00,74,75,9.22,TO,1611025426
...,...,...,...,...,...,...,...,...,...,...
573,573,Tateyama,34.9833,139.8667,48.99,53,20,12.66,JP,1611025474
574,574,Itanhém,-17.1664,-40.3300,69.46,92,19,5.14,BR,1611025474
575,575,Mareeba,-17.0000,145.4333,86.00,66,96,6.91,AU,1611025381
576,576,São Miguel,-5.4667,-35.3667,80.60,78,40,9.22,BR,1611025474


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(g_key)
locations = city_weather[["Lat", "Lng"]]
humidity = city_weather["Humidity"].astype(int)

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#finding cities with less than 90 but greater than 70 degree weather with no clouds and less than 10 mph wind speed.
degrees = city_weather.loc[(city_weather["Max Temp"] < 90) & (city_weather["Max Temp"] > 70) & 
                          (city_weather["Wind Speed"] < 10) & (city_weather["Cloudiness"] == 0)]
degrees = degrees.dropna(how="any")
degrees

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,Mar del Plata,-38.0023,-57.5575,73.99,56,0,4.61,AR,1611025365
14,14,Parkes,-33.1333,148.1833,86.34,13,0,7.92,AU,1611025427
23,23,Pochutla,15.7432,-96.4661,75.99,69,0,3.18,MX,1611025428
51,51,Maceió,-9.6658,-35.7353,73.40,94,0,3.44,BR,1611025430
62,62,Nicoya,10.1483,-85.4520,78.80,69,0,9.22,CR,1611025223
126,126,Saint-Philippe,-21.3585,55.7679,79.00,65,0,4.61,RE,1611025436
130,130,Hithadhoo,-0.6000,73.0833,80.29,69,0,7.52,MV,1611025436
173,173,Fray Bentos,-33.1325,-58.2956,72.00,53,0,1.99,UY,1611025313
179,179,Thinadhoo,0.5333,72.9333,80.44,67,0,6.82,MV,1611025415
192,192,Ikorodu,6.6191,3.5041,79.30,83,0,7.72,NG,1611025441


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
#making the hotel_df variable 
hotel_df = degrees[["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""


for index, row in hotel_df.iterrows():

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    target_coordinates = f'{row["Lat"]},{row["Lng"]}'
    target_search = "hotel"
    target_radius = 5000
    target_type = "hotel"

    params = {"location": target_coordinates,"radius": target_radius,"keyword": target_search, "type": target_type, "key": g_key}
    print("----------------------")
    response = requests.get(base_url, params=params).json()
    results = response['results']

    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except:
        print("No hotels found")
        
    

C:\Users\restr\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


----------------------


C:\Users\restr\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
No hotels found
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------


In [20]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
8,Mar del Plata,AR,-38.0023,-57.5575,Hotel Las Rocas
14,Parkes,AU,-33.1333,148.1833,Hotel Gracelands
23,Pochutla,MX,15.7432,-96.4661,Hotel Posada San Jose
51,Maceió,BR,-9.6658,-35.7353,Jatiúca Hotel & Resort
62,Nicoya,CR,10.1483,-85.4520,Hotel Doña Marta
126,Saint-Philippe,RE,-21.3585,55.7679,Le four à pain
130,Hithadhoo,MV,-0.6000,73.0833,Abuharee Grand
173,Fray Bentos,UY,-33.1325,-58.2956,Gran Hotel Fray Bentos
179,Thinadhoo,MV,0.5333,72.9333,
192,Ikorodu,NG,6.6191,3.5041,Royal Kaft Hotel & Suites


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map

fig = gmaps.figure()
markers = gmaps.marker_layer(locations, hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))